In [50]:
#import all necessary labraries
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import csv
from urllib.request import Request, urlopen
import time
from multiprocessing import Process, Manager

In [ ]:
'''
THIS CODE BLOCK IS FOR TESTING TAG INPUTS.
YOU CAN USE THE BELOW TABLE LISTS FOR VERIFYING MOVIES FROM OTHER LANGUAGES
Kannada movies 2013-2024 [1,2,3,4]
Hindi Movies 2013 - 2018 [1,2,3,4], 2019-2023 [2,3,4,5]
Telugu Movies 2013 - 2019 [1,2]; 2020 -2021 [1,2,3,4]; 2022-2023[2,3,4,5]
Tamil Movies 2013-2016 [1,2], 2016-2021 [1,2,3,4], 2022-2023 [2,3,4,5]
American movies 2013-2023 [1,2,3,4]
British movies - 2014-2020 [0,1,2,3]; 2021[1,2,3,4]; 2022-2023[2,3,4,5]

'''

year = 2021
if lang == 'Hindi':
            if (year>= 2014 and year <= 2018) or year == 2020 or year == 2021:
                table_list = [1,2,3,4]
            else:
                table_list = [2,3,4,5]
url = f"https://en.wikipedia.org/wiki/List_of_Hindi_films_of_{year}"
page=requests.get(url)
soup=BeautifulSoup(page.text,'html.parser')
mov_titles = [] 
table_elements = soup.find_all("table", class_ = 'wikitable')
print(table_elements[1])
for table in table_list:
    print(table_elements[table])


# for table in table_list:
#     i_elements = table_elements[table].find_all('i')
#     for i in i_elements:
#         title = i.find("a")
#         if (title== None):
#             continue
#         else:                
#             mov_titles.append(title.text)
# print(mov_titles)


In [3]:
def scrape_mov_titles(start_range, end_range, lang):
    file_name = f"{lang}_movie_list_{start_range}_to_{end_range}"
    with open(file_name, 'w', newline = '',encoding = 'UTF8') as head:
        header = ["YEAR", "LANGUAGE", "TITLE"]
        writer1 = csv.writer(head)
        writer1.writerow(header)
    for i in range(start_range,end_range):
        year = i
        if year<2014 and year>2023:
            print("INVALID YEAR RANGE")
        if lang.lower() not in('americanbritishhindi'):
            print('INVALID LANGUAGE SELECTED')
        if lang == 'Hindi':
            if (year>= 2014 and year <= 2018) or year == 2020 or year == 2021:
                table_list = [1,2,3,4]
            else:
                table_list = [2,3,4,5]
        if lang == 'American':
            table_list = [1,2,3,4]
        if lang == 'British':
            if year>=2014 and year<= 2020:
                table_list = [0,1,2,3]
            elif year == 2021:
                table_list = [1,2,3,4]
            else:
                table_list = [2,3,4,5]
        print(year, 'Success')

        url = f"https://en.wikipedia.org/wiki/List_of_{lang}_films_of_{year}"

      # Sending an HTTP GET request to the URL
        page=requests.get(url)
      # Parsing the HTML content using BeautifulSoup with the 'html.parser'
        soup=BeautifulSoup(page.text,'html.parser') 
        table_elements = soup.find_all("table", class_ = 'wikitable')
        for table in table_list:
            i_elements = table_elements[table].find_all('i')
            for i in i_elements:
                title = i.find("a")
                if (title== None):
                    continue
                else:
                    with open(file_name, 'a+', newline = '', encoding = 'UTF8') as rows:
                        individual_row_data = [year, lang, title.text]
                        writer2 = csv.writer(rows)
                        writer2.writerow(individual_row_data)


In [297]:
scrape_mov_titles(2014, 2024, 'British')

2014 Success
2015 Success
2016 Success
2017 Success
2018 Success
2019 Success
2020 Success
2021 Success
2022 Success
2023 Success


In [298]:
scrape_mov_titles(2014,2024,'American')

2014 Success
2015 Success
2016 Success
2017 Success
2018 Success
2019 Success
2020 Success
2021 Success
2022 Success
2023 Success


In [299]:
scrape_mov_titles(2014,2024,'Hindi')

2014 Success
2015 Success
2016 Success
2017 Success
2018 Success
2019 Success
2020 Success
2021 Success
2022 Success
2023 Success


In [6]:
mov_raw = pd.read_excel(r"C:\Users\aniru\OneDrive\Documents\SRC\SRC 4\All_movies_2013-2023.xlsx")
title_series = pd.Series(mov_raw.loc[:,'TITLE'])

In [7]:
mov_raw.iloc[23:5000:500]

,YEAR,LANGUAGE,TITLE
23,2014,American,The Monuments Men
523,2016,American,Characterz
1023,2018,American,Hell House LLC II: The Abaddon Hotel
1523,2020,American,The Boys in the Band
2023,2022,American,The Hyperions
2523,2023,American,Fair Play
3023,2017,Hindi,Mantostaan
3523,2022,Hindi,Gehraiyaan
4023,2016,British,The Pass
4523,2019,British,The Man Who Killed Don Quixote


In [35]:
def url_extractor(title):
    title_form = title.strip().replace(' ','%20')
    site= f"https://www.imdb.com/find/?q={title_form}&ref_=nv_sr_sm"
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(site,headers=hdr)
    page = urlopen(req)
    soup2 = BeautifulSoup(page, 'html.parser')
    res = soup2.find_all('a', class_ = 'ipc-metadata-list-summary-item__t', role = 'button')
    movie_link = f"https://www.imdb.com{res[0]['href']}"
    imdb_id = res_brk_dwn = res[0]['href'].split('/')[2][2:]
    return movie_link, imdb_id

In [8]:
print(url_extractor('The Monuments Men'))

('https://www.imdb.com/title/tt2177771/?ref_=fn_al_tt_1', '2177771')


Data to collect:
genres,overview,

In [36]:
def get_cast_genre(movie_url):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(movie_url,headers=hdr)
    page = urlopen(req)
    soup3 = BeautifulSoup(page, 'html.parser')
    a1_tags = soup3.find_all('a', attrs = {'data-testid':'title-cast-item__actor'})
    cast = ''
    genre_list = []

    for a in a1_tags:
        cast += a.text + ';'
    span_tags = soup3.find_all('span', attrs = {'class':'ipc-chip__text'})
    for spn in span_tags:
        genre_list += spn
    genre = ';'.join(genre_list[:-1])
    return cast, genre


In [21]:
get_cast_genre('https://www.imdb.com/title/tt10733228/?ref_=fn_al_tt_1')

('Deepika Padukone;Dhairya Karwa;Naseeruddin Shah;Ananya Panday;Rajat Kapoor;Siddhant Chaturvedi;Kanupriya Gupta;Shehzad Shaikh;Shataf Figar;Aarti Desai;Pavleen Gujral;Natasha Rastogi;Sunil Borkar;Ashish Singh;Neha Vyaso;Kamal Adib;Kavita Gupta;Karan Sharma Sanjeev;',
 'Drama;Romance')

In [37]:
def get_crew(movie_id):
    crew = ''
    crew_url = "https://www.imdb.com/title/tt"+movie_id+"/fullcredits/?ref_=tt_cl_sm"
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(crew_url,headers=hdr)
    page = urlopen(req)
    soup4 = BeautifulSoup(page, 'html.parser')
    data = soup4.find_all('td', class_='name')
    for name in data:
        crew += str(name.text).strip()+';'
    return crew

In [92]:
get_crew('2177771')

George Clooney;George Clooney;Grant Heslov;Robert M. Edsel;Bret Witter;George Clooney;Christoph Fisser;Barbara A. Hall;Grant Heslov;Henning Molfenter;Charlie Woebcken;Alexandre Desplat;Phedon Papamichael;Stephen Mirrione;Simone Bär;Luise Eigner;Jina Jay;Jim Bissell;Helen Jarvis;John King;Cornelia Ott;David Scheunemann;Bernhard Henrich;Louise Frogley;Morgane Bernhard;Christine Beveridge;Kay Georgiou;Andrea Gotowtschikow;Jan Kempkens-Odemski;Marilyn MacDonald;Andrea Pirchner;Waldo Sanchez;Petra Schaumann;Valeska Schitthelm;Daniela Skala;Heba Thorisdottir;Julia Baumann;Julia Böhm;Anja Daum;Anna Sophie Evenkamp;Sylvia Grave;Hanna Hackbeil;Jessica Haupt;Anna Jäger;Doreen Kindler;Isabell Kintzel;Julia Lechner;Winnie Mattheus;Grete Pfleger;Andrea Pirchner;Chris Rossa;Mareike Saß;Anne Schneider;Jenna Smith;Marie Sommer;Elise Tillmann;Sam Breckman;Gary Bunn;Barbara A. Hall;Michelle Lankwarden;Daniel Mattig;Arno Neubauer;Jason Nightingale;Jessie Thiele Schroeder;Pete Corral;Dennis Becker;Carlos 

In [38]:
def get_keywords(movie_id):
    keywords = ''
    key_url = "https://www.imdb.com/title/tt"+movie_id+"/keywords/?ref_=tt_stry_kw"
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(key_url,headers=hdr)
    page = urlopen(req)
    soup5 = BeautifulSoup(page, 'html.parser')
    key_data = soup5.find_all('a', class_='ipc-metadata-list-summary-item__t')
    for key in key_data:
        keywords+=key.text+';'
    return keywords

In [10]:
get_keywords('2177771')

art;art expert;nazi stolen art;1940s;world war two;christmas;1970s;year 1977;year 1945;year 1943;year 1944;mine;map;cigarette smoking;child sniper;burning painting;stolen art;stolen gold;reference to michelangelo;plundering;british army;u.s. army;paris france;repatriation;religious object;sculpture;art archive;church;fugitive;war criminal;architecture;painting;sniper;art theft;statue;nazi germany;france;resistance;art gallery;salt mine;vandalism;looting;plunder;american abroad;actor director writer;based on true story;based on book;russian soldier;red army;u.s. soldier;


In [68]:
def get_overview(movie_id):
    overview_url = "https://www.imdb.com/title/tt"+movie_id+"/plotsummary/?ref_=tt_stry_pl"
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(overview_url,headers=hdr)
    page = urlopen(req)
    soup6 = BeautifulSoup(page, 'html.parser')
    overview_data = soup6.find_all('div', class_='ipc-html-content-inner-div')
    ovw = overview_data[2].text.replace(' ',';')
    return ovw
    


In [69]:
get_overview('2177771')

"Based;on;the;true;story;of;the;greatest;treasure;hunt;in;history,;The;Monuments;Men;is;an;action;drama;focusing;on;an;unlikely;World;War;II;platoon,;tasked;by;FDR;with;going;into;Germany;to;rescue;artistic;masterpieces;from;German;thieves;and;returning;them;to;their;rightful;owners.;It;would;be;an;impossible;mission:;with;the;art;trapped;behind;enemy;lines,;and;with;the;German;Army;under;orders;to;destroy;everything;as;the;Reich;fell,;how;could;these;guys;-;seven;museum;directors,;curators,;and;art;historians,;all;more;familiar;with;Michelangelo;than;the;M1;-;possibly;hope;to;succeed?;But;as;the;Monuments;Men,;as;they;were;called,;found;themselves;in;a;race;against;time;to;avoid;the;destruction;of;1000;years;of;culture,;they;would;risk;their;lives;to;protect;and;defend;mankind's;greatest;achievements.—Sony;Pictures;Entertainment"

In [ ]:
#     with open('movie_dataset_new_post2013_test', 'w', newline = '',encoding = 'UTF8') as head:
#         header = ["id", "keywords", "genres", "original_title", "overview", "cast" , "crew"]
#         writer1 = csv.writer(head)
#         writer1.writerow(header)
#         with open('movie_dataset_new_post2013_test', 'a+', newline = '', encoding = 'UTF8') as rows:
#             individual_row_data = [mov_id, mov_kw, mov_gnr, mov_title, mov_ovw, mov_cast, mov_crew]
#             writer2 = csv.writer(rows)
#             writer2.writerow(individual_row_data)

In [48]:
test_titles1 = title_series[23:2500:500]
test_titles2 = title_series[24:2500:500]
test_titles3 = title_series[25:2500:500]
test_titles4 = title_series[26:2500:500]
test_titles5 = title_series[27:2500:500]

In [56]:
def scraper_test(test_tile_series, shared_df):
    start_time = time.time()
    cols = ['id' , 'keywords' , 'genres' , 'original_title' , 'overview' , 'cast' , 'crew']
    new_df = pd.DataFrame(columns=cols)
    for title in test_titles:
        mov_title = title
        movie_url, mov_id = url_extractor(title)
        mov_cast , mov_gnr = get_cast_genre(movie_url)
        mov_kw = get_keywords(mov_id)
        mov_crew = get_crew(mov_id)
        mov_ovw = get_overview(mov_id)
        print(mov_ovw)
#         new_row = {'id':mov_id, 'keywords': mov_kw, 'genres': mov_gnr, 'original_title': mov_title, 'overview': mov_ovw, \
#                    'cast': mov_cast, 'crew': mov_crew}
        new_df.loc[len(new_df)] = [mov_id, mov_kw, mov_gnr, mov_title, mov_ovw, mov_cast, mov_crew]
#         new_df = pd.concat([new_df, new_row])
    shared_df.append(temp_df, ignore_index = True)
    end_time = time.time()
    time_taken = end_time - start_time
    print(f"Time taken: {time_taken} seconds")

In [59]:
def main():
    # Create a Manager object
    with Manager() as manager:
        shared_df = manager.list()
        arguments = [test_titles1, test_titles2, test_titles3, test_titles4, test_titles5]
        processes = []

        # Create and start a process for each instance
        for arg in arguments:
            process = Process(target=scraper_test, args=(arg, shared_df))
            processes.append(process)
            process.start()

        # Wait for all processes to complete
        for process in processes:
            process.join()

        # Convert the list of DataFrames to a single DataFrame
        final_df = pd.concat(shared_df, ignore_index=True)
        print(final_df)


In [60]:
if __name__ == '__main__':
    main()

ValueError: No objects to concatenate